# Azure ML Compute Targets 

# Работа с Azure ML Environments

__Цель лабораторной работы:__

- научиться создавать Целевые объекты вычислений (Compute Targets) в Azure ML
- научиться Целевые объекты вычислений в Экспериментах.

## Подготовка среды

Импорт необходимых модулей и проверка версии Azure ML SDK:

In [20]:
import azureml.core
from azureml.core import Workspace, Environment, Experiment, Model
from azureml.core.conda_dependencies import CondaDependencies
from azureml.train.estimator import Estimator
from azureml.train.sklearn import SKLearn
from azureml.widgets import RunDetails

# Check core SDK version number
print(f'SDK version: {azureml.core.VERSION}')

SDK version: 1.14.0


Получим конфигурацию эксперимента: 

In [21]:
%run core.py

config = get_experiment_config('lab_5B')
init_experiment(config)
experiment_dir = get_experiment_dir(config)

config

Experiment compute-targets-experiment was initialized successfully.


{'experiment_name': 'compute-targets-experiment',
 'working_subdir': 'compute-targets-experiment-lab',
 'environment_name': 'environments-experiment',
 'core': {'expriments_root_dir': 'experiments/',
  'datastore_name': 'aml_ws_datastore_v2',
  'dataset_name': 'diabetes-data',
  'ml_cluster_name': 'aml-ws-cluster',
  'ml_model_name': 'diabetes-predict-model'}}

## Соединение со Azure ML Workspace

Устанавливаем соединение с Рабочей областью в Azure ML:

In [22]:
ws = Workspace.from_config()
print(f'Successfully connected to Workspace: {ws.name}.')

Successfully connected to Workspace: aml-workshop.


## Создаем Целевой объект вычислений

Azure ML поддерживает ряд Целевых объектов вычислений, которые можно определить в своем Рабочем пространстве и использовать для выполнения Экспериментов. При этом оплата ресурсов производится только при их использовании.

In [23]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = config['core']['ml_cluster_name']
print(f'New ML сluster name {cluster_name}')

try:
    # First let's check that the cluster doesn't exist
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print(f'The cluster {cluster.name} already exists.')
except ComputeTargetException:
    # If the cluster doesn't exist, then create it
    try:
        cluster_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', min_nodes=1, max_nodes=3)
        cluster = ComputeTarget.create(ws, cluster_name, cluster_config)
        cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

New ML сluster name aml-ws-cluster
Creating
Succeeded....................
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


Проверим статус созданного Вычислительного кластера:

In [7]:
cluster_state = cluster.get_status()

print(cluster_state.allocation_state, cluster_state.current_node_count, cluster_state.vm_size, cluster_state.creation_time)

Steady 1 STANDARD_D2_V2 2020-09-29 14:56:06.376303+00:00


## Запуск обучения ML модели на ML кластере

Запустим процесс обучения модели машинного обучения на только что созданном кластере.

### Подготовка

Используем [ранее загруженный](02B-datastores-and-datasets.ipynb) датасет в качестве __данных для обучения__: 

In [8]:
data_ds = ws.datasets.get(config['core']['dataset_name'])
print(f'Used dataset {data_ds.name}: {data_ds.description}')

Used dataset diabetes-data: Diabetes Disease Database


Используем [ранее зарегистрированную](05A-environments.ipynb) Среду вычисления в качестве __Environment для обучения модели__:

In [9]:
env = Environment.get(ws, config['environment_name'])
print(f'Environment {env.name} will be used.')

Environment environments-experiment will be used.


Получим скрипт обучения модели:

In [11]:
!cp scripts/train_model.py $experiment_dir
!ls $experiment_dir

train_model.py


Создадим Estimator, передав в качестве параметра `compute_target` имя созданного Вычислительного кластера:

In [12]:
estimator = SKLearn(source_directory=experiment_dir,
                    inputs=[data_ds.as_named_input('data')],
                    entry_script='train_model.py',
                    script_params={'--reg_rate': 0.1},
                    compute_target=cluster_name, # Use the compute target created previously
                    environment_definition=env
                    )

WARNING - If environment_definition or conda_dependencies_file_path is specified, Azure ML will not install any framework related packages on behalf of the user.


Создаем Эксперимент и запускаем обучение модели:

In [ ]:
# Create and run the experiment
experiment = Experiment(workspace=ws, name=config['experiment_name'])
run = experiment.submit(config=estimator)

# Get run details
run.wait_for_completion(show_output=True)
RunDetails(run).show()

Посмотрите на результаты обучения модели на портале Azure ML, перейдя по ссылке выше.

## Результаты

Просмотрим результаты обучения модели с использованием нового Environment:

In [14]:
metrics = run.get_metrics()

for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')

for file in run.get_file_names():
    print(file)

Regularization Rate 0.1
Accuracy 0.787
AUC 0.8576173518134779
ROC aml://artifactId/ExperimentRun/dcid.compute-targets-experiment_1601391683_6915ee90/ROC_1601392409.png


ROC_1601392409.png
azureml-logs/20_image_build_log.txt
azureml-logs/55_azureml-execution-tvmps_726f40933c29105db6d8ab7b615efb7ba63760662dbf5a2da36d48a073206484_d.txt
azureml-logs/65_job_prep-tvmps_726f40933c29105db6d8ab7b615efb7ba63760662dbf5a2da36d48a073206484_d.txt
azureml-logs/70_driver_log.txt
azureml-logs/75_job_post-tvmps_726f40933c29105db6d8ab7b615efb7ba63760662dbf5a2da36d48a073206484_d.txt
azureml-logs/process_info.json
azureml-logs/process_status.json
logs/azureml/107_azureml.log
logs/azureml/dataprep/backgroundProcess.log
logs/azureml/dataprep/backgroundProcess_Telemetry.log
logs/azureml/dataprep/engine_spans_fda35e16-cf16-4879-b9ff-7820544b5461.jsonl
logs/azureml/dataprep/python_span_22d683d9-72f8-4b9c-a222-e85dcc3a5851.jsonl
logs/azureml/dataprep/python_span_fda35e16-cf16-4879-b9ff-7820544b5461.jsonl
logs/a

## Регистрация модели

Зарегистрируем обученную модель:

In [15]:
run.register_model(model_path=run.get_file_names()[-1], 
                   model_name=config['core']['ml_model_name'],
                   tags={'Lab':'5B'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

Model(workspace=Workspace.create(name='aml-workshop', subscription_id='9aef4ce1-e591-4870-9443-0b0eb98df2aa', resource_group='ai-in-cloud-workshop-rg'), name=diabetes-predict-model, id=diabetes-predict-model:2, version=2, tags={'Lab': '5B'}, properties={'AUC': '0.8576173518134779', 'Accuracy': '0.787'})

Получим список всех зарегистрированных ML моделей:

In [17]:
for model in Model.list(ws):
    print(f'{model.name} v{model.version}')
    
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print(f'\t {tag_name}: {tag}')
        
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print(f'\t {prop_name}: {prop}')
        
    print('\n')

diabetes-predict-model v2
	 Lab: 5B
	 AUC: 0.8576173518134779
	 Accuracy: 0.787


diabetes-predict-model v1
	 Lab: 4B
	 AUC: 0.8576982541474856
	 Accuracy: 0.7876666666666666




Проcмотрите зарегистрированную модель на портале Azure ML в разделе: `<your_workspace> > Models > diabetes_model`.